In [1]:
import os
import time
import random
import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
import optuna
from augraphy import AugraphyPipeline, PaperFactory, InkBleed, Brightness, SubtleNoise, NoiseTexturize, Gamma



In [12]:
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
# Dataset class
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

In [14]:
# Function to train one epoch
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()
        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

    return ret


In [15]:
# Function to evaluate the model
def evaluate(loader, model, loss_fn, device):
    model.eval()
    val_loss = 0
    preds_list = []
    targets_list = []

    with torch.no_grad():
        for image, targets in loader:
            image = image.to(device)
            targets = targets.to(device)

            preds = model(image)
            loss = loss_fn(preds, targets)

            val_loss += loss.item()
            preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(targets.detach().cpu().numpy())

    val_loss /= len(loader)
    val_acc = accuracy_score(targets_list, preds_list)
    val_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "val_loss": val_loss,
        "val_acc": val_acc,
        "val_f1": val_f1,
    }

    return ret


In [16]:
# Augraphy pipeline
aug_pipeline = AugraphyPipeline([
    PaperFactory(p=1),
    InkBleed(p=0.5),
    Brightness(p=0.5),
    SubtleNoise(p=0.5),
    NoiseTexturize(p=0.5),
    Gamma(p=0.5),
])

In [17]:

def apply_augraphy(image, **kwargs):
    if isinstance(image, np.ndarray):
        image = aug_pipeline(image=image)['image']
    return image

# Function to define data transforms
def get_transforms(img_size):
    trn_transform = A.Compose([
        A.Resize(height=img_size, width=img_size),
        A.Lambda(image=apply_augraphy),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Rotate(limit=30, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

    tst_transform = A.Compose([
        A.Resize(height=img_size, width=img_size),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

    return trn_transform, tst_transform

In [18]:
def objective(trial):
    # Hyperparameters
    img_size = trial.suggest_categorical("img_size", [224, 256, 380])
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-3)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    epochs = trial.suggest_int("epochs", 5, 20)

    # Get data transforms
    trn_transform, tst_transform = get_transforms(img_size)

    # Dataset and DataLoader
    trn_dataset = ImageDataset("dj/data/train.csv", "dj/data/train/", transform=trn_transform)
    tst_dataset = ImageDataset("dj/data/sample_submission.csv", "dj/data/test/", transform=tst_transform)

    trn_loader = DataLoader(trn_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
    tst_loader = DataLoader(tst_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

    # Load model
    model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=17).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=lr)

    # Early stopping and learning rate scheduler
    early_stopping_patience = 3
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2)

    best_val_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(epochs):
        train_metrics = train_one_epoch(trn_loader, model, optimizer, loss_fn, device)
        val_metrics = evaluate(tst_loader, model, loss_fn, device)

        scheduler.step(val_metrics['val_loss'])

        if val_metrics['val_loss'] < best_val_loss:
            best_val_loss = val_metrics['val_loss']
            epochs_no_improve = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= early_stopping_patience:
            break

    model.load_state_dict(torch.load('best_model.pth'))
    final_metrics = evaluate(tst_loader, model, loss_fn, device)

    return final_metrics['val_f1']

In [19]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

# Print the best hyperparameters
print("Best hyperparameters: ", study.best_params)

# Load the best model for final evaluation
model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=17).to(device)
model.load_state_dict(torch.load('best_model.pth'))

[I 2024-07-31 12:16:47,668] A new study created in memory with name: no-name-7ea6e2a5-614f-4785-8ecb-cc63f346999e
INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/efficientnet_b4.ra2_in1k)
INFO:timm.models._hub:[timm/efficientnet_b4.ra2_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
INFO:timm.models._builder:Missing keys (classifier.weight, classifier.bias) discovered while loading pretrained weights. This is expected if model is being adapted.
  0%|          | 0/25 [00:00<?, ?it/s]
[W 2024-07-31 12:16:49,012] Trial 0 failed with parameters: {'img_size': 224, 'lr': 0.0002566692858472912, 'batch_size': 64, 'epochs': 9} because of the following error: IndexError('only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices').
Traceback (most recent call last):
  File "/opt/conda/envs/cv/lib/python3.10/site-packages/optuna/study/_o

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [20]:
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Dataset class
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

# Function to train one epoch
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()
        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

    return ret

# Function to evaluate the model
def evaluate(loader, model, loss_fn, device):
    model.eval()
    val_loss = 0
    preds_list = []
    targets_list = []

    with torch.no_grad():
        for image, targets in loader:
            image = image.to(device)
            targets = targets.to(device)

            preds = model(image)
            loss = loss_fn(preds, targets)

            val_loss += loss.item()
            preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(targets.detach().cpu().numpy())

    val_loss /= len(loader)
    val_acc = accuracy_score(targets_list, preds_list)
    val_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "val_loss": val_loss,
        "val_acc": val_acc,
        "val_f1": val_f1,
    }

    return ret

# Augraphy pipeline
aug_pipeline = AugraphyPipeline([
    PaperFactory(p=1),
    InkBleed(p=0.5),
    Brightness(p=0.5),
    SubtleNoise(p=0.5),
    NoiseTexturize(p=0.5),
    Gamma(p=0.5),
])

def apply_augraphy(image, **kwargs):
    if isinstance(image, np.ndarray):
        image = aug_pipeline(image=image)['image']
    return image

# Function to define data transforms
def get_transforms(img_size):
    trn_transform = A.Compose([
        A.Resize(height=img_size, width=img_size),
        A.Lambda(image=apply_augraphy),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Rotate(limit=30, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

    tst_transform = A.Compose([
        A.Resize(height=img_size, width=img_size),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

    return trn_transform, tst_transform

# Optuna objective function
def objective(trial):
    # Hyperparameters
    img_size = trial.suggest_categorical("img_size", [224, 256, 380])
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-3)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    epochs = trial.suggest_int("epochs", 5, 20)

    # Get data transforms
    trn_transform, tst_transform = get_transforms(img_size)

    # Dataset and DataLoader
    trn_dataset = ImageDataset("d/train.csv", "d/train/", transform=trn_transform)
    tst_dataset = ImageDataset("d/sample_submission.csv", "d/test/", transform=tst_transform)

    trn_loader = DataLoader(trn_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
    tst_loader = DataLoader(tst_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

    # Load model
    model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=17).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=lr)

    # Early stopping and learning rate scheduler
    early_stopping_patience = 3
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2)

    best_val_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(epochs):
        train_metrics = train_one_epoch(trn_loader, model, optimizer, loss_fn, device)
        val_metrics = evaluate(tst_loader, model, loss_fn, device)

        scheduler.step(val_metrics['val_loss'])

        if val_metrics['val_loss'] < best_val_loss:
            best_val_loss = val_metrics['val_loss']
            epochs_no_improve = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= early_stopping_patience:
            break

    model.load_state_dict(torch.load('best_model.pth'))
    final_metrics = evaluate(tst_loader, model, loss_fn, device)

    return final_metrics['val_f1']

# Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

# Print the best hyperparameters
print("Best hyperparameters: ", study.best_params)

# Load the best model for final evaluation
model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=17).to(device)
model.load_state_dict(torch.load('best_model.pth'))

[I 2024-07-31 12:18:28,316] A new study created in memory with name: no-name-d7515dd4-485c-419c-911a-150a79c37879
INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/efficientnet_b4.ra2_in1k)
INFO:timm.models._hub:[timm/efficientnet_b4.ra2_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
INFO:timm.models._builder:Missing keys (classifier.weight, classifier.bias) discovered while loading pretrained weights. This is expected if model is being adapted.
  0%|          | 0/25 [00:00<?, ?it/s]
[W 2024-07-31 12:18:29,553] Trial 0 failed with parameters: {'img_size': 256, 'lr': 2.2989604071238654e-05, 'batch_size': 64, 'epochs': 10} because of the following error: IndexError('only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices').
Traceback (most recent call last):
  File "/opt/conda/envs/cv/lib/python3.10/site-packages/optuna/study/

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

[I 2024-07-31 12:21:56,268] A new study created in memory with name: no-name-94aa9201-d031-456b-98ed-80918777037d
INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/efficientnet_b4.ra2_in1k)
INFO:timm.models._hub:[timm/efficientnet_b4.ra2_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
INFO:timm.models._builder:Missing keys (classifier.weight, classifier.bias) discovered while loading pretrained weights. This is expected if model is being adapted.
  0%|          | 0/50 [00:00<?, ?it/s]

Original image shape: (224, 224, 3), dtype: uint8
Formatted image shape: (224, 224, 3), dtype: uint8


  0%|          | 0/50 [00:00<?, ?it/s]
[W 2024-07-31 12:21:57,592] Trial 0 failed with parameters: {'img_size': 224, 'lr': 0.00021573834878686518, 'batch_size': 32, 'epochs': 14} because of the following error: IndexError('only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices').
Traceback (most recent call last):
  File "/opt/conda/envs/cv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_9488/3944791630.py", line 190, in objective
    train_metrics = train_one_epoch(trn_loader, model, optimizer, loss_fn, device)
  File "/tmp/ipykernel_9488/3944791630.py", line 58, in train_one_epoch
    for image, targets in pbar:
  File "/opt/conda/envs/cv/lib/python3.10/site-packages/tqdm/std.py", line 1181, in __iter__
    for obj in iterable:
  File "/opt/conda/envs/cv/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 630, in __nex

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [25]:
import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
import optuna
from augraphy import AugraphyPipeline, PaperFactory, InkBleed, Brightness, SubtleNoise, NoiseTexturize, Gamma

# Set seed for reproducibility
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Dataset class
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)).convert('RGB'))  # Ensure RGB
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

# Function to train one epoch
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()
        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

    return ret

# Function to evaluate the model
def evaluate(loader, model, loss_fn, device):
    model.eval()
    val_loss = 0
    preds_list = []
    targets_list = []

    with torch.no_grad():
        for image, targets in loader:
            image = image.to(device)
            targets = targets.to(device)

            preds = model(image)
            loss = loss_fn(preds, targets)

            val_loss += loss.item()
            preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(targets.detach().cpu().numpy())

    val_loss /= len(loader)
    val_acc = accuracy_score(targets_list, preds_list)
    val_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "val_loss": val_loss,
        "val_acc": val_acc,
        "val_f1": val_f1,
    }

    return ret

# Augraphy pipeline
aug_pipeline = AugraphyPipeline([
    PaperFactory(p=1),
    InkBleed(p=0.5),
    Brightness(p=0.5),
    SubtleNoise(p=0.5),
    NoiseTexturize(p=0.5),
    Gamma(p=0.5),
])

def apply_augraphy(image, **kwargs):
    if isinstance(image, np.ndarray):
        print(f"Original image shape: {image.shape}, dtype: {image.dtype}")  # Debugging statement
        if image.ndim == 2:  # if grayscale, convert to RGB
            image = np.stack([image] * 3, axis=-1)
        elif image.shape[2] == 1:  # if single channel, convert to RGB
            image = np.repeat(image, 3, axis=2)
        print(f"Formatted image shape: {image.shape}, dtype: {image.dtype}")  # Debugging statement
        try:
            aug_image = aug_pipeline(image=image)
            if isinstance(aug_image, dict) and 'image' in aug_image:
                image = aug_image['image']
            elif isinstance(aug_image, np.ndarray):
                image = aug_image
            else:
                raise ValueError("Unexpected output from Augraphy pipeline.")
        except Exception as e:
            print(f"Error applying Augraphy pipeline: {e}")
    return image

# Function to define data transforms
def get_transforms(img_size):
    trn_transform = A.Compose([
        A.Resize(height=img_size, width=img_size),
        A.Lambda(image=apply_augraphy),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Rotate(limit=30, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

    tst_transform = A.Compose([
        A.Resize(height=img_size, width=img_size),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

    return trn_transform, tst_transform

# Optuna objective function
def objective(trial):
    # Hyperparameters
    img_size = trial.suggest_categorical("img_size", [224, 256, 380])
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-3)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    epochs = trial.suggest_int("epochs", 5, 20)

    # Get data transforms
    trn_transform, tst_transform = get_transforms(img_size)

    # Dataset and DataLoader
    trn_dataset = ImageDataset("dj/data/train.csv", "dj/data/train/", transform=trn_transform)
    tst_dataset = ImageDataset("dj/data/sample_submission.csv", "dj/data/test/", transform=tst_transform)

    trn_loader = DataLoader(trn_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
    tst_loader = DataLoader(tst_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

    # Load model
    model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=17).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=lr)

    # Early stopping and learning rate scheduler
    early_stopping_patience = 3
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2)

    best_val_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(epochs):
        train_metrics = train_one_epoch(trn_loader, model, optimizer, loss_fn, device)
        val_metrics = evaluate(tst_loader, model, loss_fn, device)

        scheduler.step(val_metrics['val_loss'])

        if val_metrics['val_loss'] < best_val_loss:
            best_val_loss = val_metrics['val_loss']
            epochs_no_improve = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= early_stopping_patience:
            break

    model.load_state_dict(torch.load('best_model.pth'))
    final_metrics = evaluate(tst_loader, model, loss_fn, device)

    return final_metrics['val_f1']

# Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

# Print the best hyperparameters
print("Best hyperparameters: ", study.best_params)

# Load the best model for final evaluation
model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=17).to(device)
model.load_state_dict(torch.load('best_model.pth'))


[I 2024-07-31 12:27:46,579] A new study created in memory with name: no-name-d50d5bf4-1fd6-406e-aff4-897825837235


INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/efficientnet_b4.ra2_in1k)
INFO:timm.models._hub:[timm/efficientnet_b4.ra2_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
INFO:timm.models._builder:Missing keys (classifier.weight, classifier.bias) discovered while loading pretrained weights. This is expected if model is being adapted.
  0%|          | 0/50 [00:00<?, ?it/s]

Original image shape: (224, 224, 3), dtype: uint8
Formatted image shape: (224, 224, 3), dtype: uint8
Original image shape: (224, 224, 3), dtype: uint8
Formatted image shape: (224, 224, 3), dtype: uint8
Original image shape: (224, 224, 3), dtype: uint8
Formatted image shape: (224, 224, 3), dtype: uint8
Original image shape: (224, 224, 3), dtype: uint8
Formatted image shape: (224, 224, 3), dtype: uint8
Original image shape: (224, 224, 3), dtype: uint8
Formatted image shape: (224, 224, 3), dtype: uint8
Original image shape: (224, 224, 3), dtype: uint8
Formatted image shape: (224, 224, 3), dtype: uint8
Original image shape: (224, 224, 3), dtype: uint8
Formatted image shape: (224, 224, 3), dtype: uint8
Original image shape: (224, 224, 3), dtype: uint8
Formatted image shape: (224, 224, 3), dtype: uint8
Original image shape: (224, 224, 3), dtype: uint8
Formatted image shape: (224, 224, 3), dtype: uint8
Original image shape: (224, 224, 3), dtype: uint8
Formatted image shape: (224, 224, 3), dty

  0%|          | 0/50 [00:15<?, ?it/s]
[W 2024-07-31 12:28:07,725] Trial 0 failed with parameters: {'img_size': 224, 'lr': 1.034783317808543e-05, 'batch_size': 32, 'epochs': 8} because of the following error: RuntimeError('stack expects each tensor to be equal size, but got [3, 224, 224] at entry 0 and [3, 224, 213] at entry 2').
Traceback (most recent call last):
  File "/opt/conda/envs/cv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_9488/1456837990.py", line 199, in objective
    train_metrics = train_one_epoch(trn_loader, model, optimizer, loss_fn, device)
  File "/tmp/ipykernel_9488/1456837990.py", line 58, in train_one_epoch
    for image, targets in pbar:
  File "/opt/conda/envs/cv/lib/python3.10/site-packages/tqdm/std.py", line 1181, in __iter__
    for obj in iterable:
  File "/opt/conda/envs/cv/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 630, in __next__
    data

RuntimeError: stack expects each tensor to be equal size, but got [3, 224, 224] at entry 0 and [3, 224, 213] at entry 2

In [27]:
import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
import optuna
from augraphy import AugraphyPipeline, PaperFactory, InkBleed, Brightness, SubtleNoise, NoiseTexturize, Gamma

# Set seed for reproducibility
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Dataset class
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)).convert('RGB'))  # Ensure RGB
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

# Function to train one epoch
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()
        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

    return ret

# Function to evaluate the model
def evaluate(loader, model, loss_fn, device):
    model.eval()
    val_loss = 0
    preds_list = []
    targets_list = []

    with torch.no_grad():
        for image, targets in loader:
            image = image.to(device)
            targets = targets.to(device)

            preds = model(image)
            loss = loss_fn(preds, targets)

            val_loss += loss.item()
            preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(targets.detach().cpu().numpy())

    val_loss /= len(loader)
    val_acc = accuracy_score(targets_list, preds_list)
    val_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "val_loss": val_loss,
        "val_acc": val_acc,
        "val_f1": val_f1,
    }

    return ret

# Augraphy pipeline
aug_pipeline = AugraphyPipeline([
    PaperFactory(p=1),
    InkBleed(p=0.5),
    Brightness(p=0.5),
    SubtleNoise(p=0.5),
    NoiseTexturize(p=0.5),
    Gamma(p=0.5),
])

def apply_augraphy(image, **kwargs):
    if isinstance(image, np.ndarray):
        print(f"Original image shape: {image.shape}, dtype: {image.dtype}")  # Debugging statement
        if image.ndim == 2:  # if grayscale, convert to RGB
            image = np.stack([image] * 3, axis=-1)
        elif image.shape[2] == 1:  # if single channel, convert to RGB
            image = np.repeat(image, 3, axis=2)
        print(f"Formatted image shape: {image.shape}, dtype: {image.dtype}")  # Debugging statement
        try:
            aug_image = aug_pipeline(image=image)
            if isinstance(aug_image, dict) and 'image' in aug_image:
                image = aug_image['image']
            elif isinstance(aug_image, np.ndarray):
                image = aug_image
            else:
                raise ValueError("Unexpected output from Augraphy pipeline.")
        except Exception as e:
            print(f"Error applying Augraphy pipeline: {e}")
    return image

# Function to define data transforms
def get_transforms(img_size):
    trn_transform = A.Compose([
        A.Resize(height=img_size, width=img_size),  # Ensure resize is applied first
        A.Lambda(image=apply_augraphy),
        A.PadIfNeeded(min_height=img_size, min_width=img_size, border_mode=0, value=0),  # Add padding to ensure consistent size
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Rotate(limit=30, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

    tst_transform = A.Compose([
        A.Resize(height=img_size, width=img_size),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

    return trn_transform, tst_transform

# Optuna objective function
def objective(trial):
    # Hyperparameters
    img_size = trial.suggest_categorical("img_size", [224, 256, 380])
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-3)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    epochs = trial.suggest_int("epochs", 5, 20)

    # Get data transforms
    trn_transform, tst_transform = get_transforms(img_size)

    # Dataset and DataLoader
    trn_dataset = ImageDataset("dj/data/train.csv", "dj/data/train/", transform=trn_transform)
    tst_dataset = ImageDataset("dj/data/sample_submission.csv", "dj/data/test/", transform=tst_transform)

    trn_loader = DataLoader(trn_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
    tst_loader = DataLoader(tst_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

    # Load model
    model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=17).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=lr)

    # Early stopping and learning rate scheduler
    early_stopping_patience = 3
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2)

    best_val_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(epochs):
        train_metrics = train_one_epoch(trn_loader, model, optimizer, loss_fn, device)
        val_metrics = evaluate(tst_loader, model, loss_fn, device)

        scheduler.step(val_metrics['val_loss'])

        if val_metrics['val_loss'] < best_val_loss:
            best_val_loss = val_metrics['val_loss']
            epochs_no_improve = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= early_stopping_patience:
            break

    model.load_state_dict(torch.load('best_model.pth'))
    final_metrics = evaluate(tst_loader, model, loss_fn, device)

    return final_metrics['val_f1']

# Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

# Print the best hyperparameters
print("Best hyperparameters: ", study.best_params)

# Load the best model for final evaluation
model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=17).to(device)
model.load_state_dict(torch.load('best_model.pth'))


[I 2024-07-31 12:31:00,362] A new study created in memory with name: no-name-b5d75167-8d9e-45b7-b98d-dc9dadd040d0
INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/efficientnet_b4.ra2_in1k)
INFO:timm.models._hub:[timm/efficientnet_b4.ra2_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
INFO:timm.models._builder:Missing keys (classifier.weight, classifier.bias) discovered while loading pretrained weights. This is expected if model is being adapted.
  0%|          | 0/25 [00:00<?, ?it/s]

Original image shape: (380, 380, 3), dtype: uint8
Formatted image shape: (380, 380, 3), dtype: uint8
Original image shape: (380, 380, 3), dtype: uint8
Formatted image shape: (380, 380, 3), dtype: uint8
Original image shape: (380, 380, 3), dtype: uint8
Formatted image shape: (380, 380, 3), dtype: uint8
Original image shape: (380, 380, 3), dtype: uint8
Formatted image shape: (380, 380, 3), dtype: uint8
Original image shape: (380, 380, 3), dtype: uint8
Formatted image shape: (380, 380, 3), dtype: uint8
Original image shape: (380, 380, 3), dtype: uint8
Formatted image shape: (380, 380, 3), dtype: uint8
Original image shape: (380, 380, 3), dtype: uint8
Formatted image shape: (380, 380, 3), dtype: uint8
Original image shape: (380, 380, 3), dtype: uint8
Formatted image shape: (380, 380, 3), dtype: uint8
Original image shape: (380, 380, 3), dtype: uint8
Formatted image shape: (380, 380, 3), dtype: uint8
Original image shape: (380, 380, 3), dtype: uint8
Formatted image shape: (380, 380, 3), dty

  0%|          | 0/25 [00:41<?, ?it/s]
[W 2024-07-31 12:31:46,766] Trial 0 failed with parameters: {'img_size': 380, 'lr': 1.3790469866821417e-05, 'batch_size': 64, 'epochs': 19} because of the following error: OutOfMemoryError('CUDA out of memory. Tried to allocate 190.00 MiB. GPU 0 has a total capacity of 23.69 GiB of which 162.81 MiB is free. Process 3618428 has 23.52 GiB memory in use. Of the allocated memory 22.96 GiB is allocated by PyTorch, and 243.36 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)').
Traceback (most recent call last):
  File "/opt/conda/envs/cv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_9488/3533208221.py", line 200, in objective
   

OutOfMemoryError: CUDA out of memory. Tried to allocate 190.00 MiB. GPU 0 has a total capacity of 23.69 GiB of which 162.81 MiB is free. Process 3618428 has 23.52 GiB memory in use. Of the allocated memory 22.96 GiB is allocated by PyTorch, and 243.36 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [1]:
import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
import optuna
from augraphy import AugraphyPipeline, PaperFactory, InkBleed, Brightness, SubtleNoise, NoiseTexturize, Gamma

# Set seed for reproducibility
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Dataset class
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)).convert('RGB'))  # Ensure RGB
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

# Function to train one epoch with gradient accumulation
def train_one_epoch(loader, model, optimizer, loss_fn, device, accumulation_steps=4):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    optimizer.zero_grad()
    pbar = tqdm(loader)
    for i, (image, targets) in enumerate(pbar):
        image = image.to(device)
        targets = targets.to(device)

        preds = model(image)
        loss = loss_fn(preds, targets) / accumulation_steps
        loss.backward()

        if (i + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        train_loss += loss.item() * accumulation_steps
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

    return ret

# Function to evaluate the model
def evaluate(loader, model, loss_fn, device):
    model.eval()
    val_loss = 0
    preds_list = []
    targets_list = []

    with torch.no_grad():
        for image, targets in loader:
            image = image.to(device)
            targets = targets.to(device)

            preds = model(image)
            loss = loss_fn(preds, targets)

            val_loss += loss.item()
            preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(targets.detach().cpu().numpy())

    val_loss /= len(loader)
    val_acc = accuracy_score(targets_list, preds_list)
    val_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "val_loss": val_loss,
        "val_acc": val_acc,
        "val_f1": val_f1,
    }

    return ret

# Augraphy pipeline
aug_pipeline = AugraphyPipeline([
    PaperFactory(p=1),
    InkBleed(p=0.5),
    Brightness(p=0.5),
    SubtleNoise(p=0.5),
    NoiseTexturize(p=0.5),
    Gamma(p=0.5),
])

def apply_augraphy(image, **kwargs):
    if isinstance(image, np.ndarray):
        print(f"Original image shape: {image.shape}, dtype: {image.dtype}")  # Debugging statement
        if image.ndim == 2:  # if grayscale, convert to RGB
            image = np.stack([image] * 3, axis=-1)
        elif image.shape[2] == 1:  # if single channel, convert to RGB
            image = np.repeat(image, 3, axis=2)
        print(f"Formatted image shape: {image.shape}, dtype: {image.dtype}")  # Debugging statement
        try:
            aug_image = aug_pipeline(image=image)
            if isinstance(aug_image, dict) and 'image' in aug_image:
                image = aug_image['image']
            elif isinstance(aug_image, np.ndarray):
                image = aug_image
            else:
                raise ValueError("Unexpected output from Augraphy pipeline.")
        except Exception as e:
            print(f"Error applying Augraphy pipeline: {e}")
    return image

# Function to define data transforms
def get_transforms(img_size):
    trn_transform = A.Compose([
        A.Resize(height=img_size, width=img_size),  # Ensure resize is applied first
        A.Lambda(image=apply_augraphy),
        A.PadIfNeeded(min_height=img_size, min_width=img_size, border_mode=0, value=0),  # Add padding to ensure consistent size
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Rotate(limit=30, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

    tst_transform = A.Compose([
        A.Resize(height=img_size, width=img_size),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

    return trn_transform, tst_transform

# Optuna objective function
def objective(trial):
    # Hyperparameters
    img_size = trial.suggest_categorical("img_size", [224, 256])  # limit to smaller sizes to prevent OOM
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-3)
    batch_size = trial.suggest_categorical("batch_size", [16, 32])
    epochs = trial.suggest_int("epochs", 5, 20)
    accumulation_steps = 4

    # Get data transforms
    trn_transform, tst_transform = get_transforms(img_size)

    # Dataset and DataLoader
    trn_dataset = ImageDataset("dj/data/train.csv", "dj/data/train/", transform=trn_transform)
    tst_dataset = ImageDataset("dj/data/sample_submission.csv", "dj/data/test/", transform=tst_transform)

    trn_loader = DataLoader(trn_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
    tst_loader = DataLoader(tst_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

    # Load model
    model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=17).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=lr)

    # Early stopping and learning rate scheduler
    early_stopping_patience = 3
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2)

    best_val_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(epochs):
        train_metrics = train_one_epoch(trn_loader, model, optimizer, loss_fn, device, accumulation_steps)
        val_metrics = evaluate(tst_loader, model, loss_fn, device)

        scheduler.step(val_metrics['val_loss'])

        if val_metrics['val_loss'] < best_val_loss:
            best_val_loss = val_metrics['val_loss']
            epochs_no_improve = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= early_stopping_patience:
            break

    model.load_state_dict(torch.load('best_model.pth'))
    final_metrics = evaluate(tst_loader, model, loss_fn, device)

    return final_metrics['val_f1']

# Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

# Print the best hyperparameters
print("Best hyperparameters: ", study.best_params)

# Load the best model for final evaluation
model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=17).to(device)
model.load_state_dict(torch.load('best_model.pth'))


[I 2024-07-31 12:35:05,657] A new study created in memory with name: no-name-9412285f-69bb-4438-9680-b507e4a0bfbc
INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/efficientnet_b4.ra2_in1k)
INFO:timm.models._hub:[timm/efficientnet_b4.ra2_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
INFO:timm.models._builder:Missing keys (classifier.weight, classifier.bias) discovered while loading pretrained weights. This is expected if model is being adapted.
  0%|          | 0/99 [00:00<?, ?it/s]

Original image shape: (256, 256, 3), dtype: uint8
Formatted image shape: (256, 256, 3), dtype: uint8
Original image shape: (256, 256, 3), dtype: uint8
Formatted image shape: (256, 256, 3), dtype: uint8
Original image shape: (256, 256, 3), dtype: uint8
Formatted image shape: (256, 256, 3), dtype: uint8
Original image shape: (256, 256, 3), dtype: uint8
Formatted image shape: (256, 256, 3), dtype: uint8
Original image shape: (256, 256, 3), dtype: uint8
Formatted image shape: (256, 256, 3), dtype: uint8
Original image shape: (256, 256, 3), dtype: uint8
Formatted image shape: (256, 256, 3), dtype: uint8
Original image shape: (256, 256, 3), dtype: uint8
Formatted image shape: (256, 256, 3), dtype: uint8
Original image shape: (256, 256, 3), dtype: uint8
Formatted image shape: (256, 256, 3), dtype: uint8
Original image shape: (256, 256, 3), dtype: uint8
Formatted image shape: (256, 256, 3), dtype: uint8
Original image shape: (256, 256, 3), dtype: uint8
Formatted image shape: (256, 256, 3), dty

Loss: 0.7628:   1%|          | 1/99 [00:10<16:48, 10.30s/it]

Original image shape: (256, 256, 3), dtype: uint8
Formatted image shape: (256, 256, 3), dtype: uint8
Original image shape: (256, 256, 3), dtype: uint8
Formatted image shape: (256, 256, 3), dtype: uint8
Original image shape: (256, 256, 3), dtype: uint8
Formatted image shape: (256, 256, 3), dtype: uint8
Original image shape: (256, 256, 3), dtype: uint8
Formatted image shape: (256, 256, 3), dtype: uint8
Original image shape: (256, 256, 3), dtype: uint8
Formatted image shape: (256, 256, 3), dtype: uint8
Original image shape: (256, 256, 3), dtype: uint8
Formatted image shape: (256, 256, 3), dtype: uint8
Original image shape: (256, 256, 3), dtype: uint8
Formatted image shape: (256, 256, 3), dtype: uint8
Original image shape: (256, 256, 3), dtype: uint8
Formatted image shape: (256, 256, 3), dtype: uint8
Original image shape: (256, 256, 3), dtype: uint8
Formatted image shape: (256, 256, 3), dtype: uint8
Original image shape: (256, 256, 3), dtype: uint8
Formatted image shape: (256, 256, 3), dty

Loss: 0.7697:   2%|▏         | 2/99 [00:21<17:55, 11.09s/it]

...

Loss: 0.7097:  98%|█████████▊| 97/99 [15:20<00:17,  8.51s/it]

Original image shape: (256, 256, 3), dtype: uint8
Formatted image shape: (256, 256, 3), dtype: uint8
Original image shape: (256, 256, 3), dtype: uint8
Formatted image shape: (256, 256, 3), dtype: uint8
Original image shape: (256, 256, 3), dtype: uint8
Formatted image shape: (256, 256, 3), dtype: uint8
Original image shape: (256, 256, 3), dtype: uint8
Formatted image shape: (256, 256, 3), dtype: uint8
Original image shape: (256, 256, 3), dtype: uint8
Formatted image shape: (256, 256, 3), dtype: uint8
Original image shape: (256, 256, 3), dtype: uint8
Formatted image shape: (256, 256, 3), dtype: uint8
Original image shape: (256, 256, 3), dtype: uint8
Formatted image shape: (256, 256, 3), dtype: uint8
Original image shape: (256, 256, 3), dtype: uint8
Formatted image shape: (256, 256, 3), dtype: uint8
Original image shape: (256, 256, 3), dtype: uint8
Formatted image shape: (256, 256, 3), dtype: uint8
Original image shape: (256, 256, 3), dtype: uint8
Formatted image shape: (256, 256, 3), dty

Loss: 0.7462:  99%|█████████▉| 98/99 [15:31<00:09,  9.29s/it]

Original image shape: (256, 256, 3), dtype: uint8
Formatted image shape: (256, 256, 3), dtype: uint8
Original image shape: (256, 256, 3), dtype: uint8
Formatted image shape: (256, 256, 3), dtype: uint8


Loss: 0.5564: 100%|██████████| 99/99 [15:32<00:00,  9.42s/it]
